In [1]:
!pip install scikit-learn --quiet

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_mnist(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # Conv1: 28x28x1 -> 28x28x6 (kernel=5, padding=2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.act1 = nn.Sigmoid()
        # Pool1: 28x28x6 -> 14x14x6 (AvgPool, kernel=2, stride=2)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

        # Conv2: 14x14x6 -> 10x10x16 (kernel=5, no pad)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.act2 = nn.Sigmoid()
        # Pool2: 10x10x16 -> 5x5x16 (AvgPool, kernel=2, stride=2)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        # Kích thước đầu vào lớp Fully Connected: 5*5*16 = 400
        self.fc_input_size = 5 * 5 * 16

        # Dense1: 400 -> 120
        self.fc1 = nn.Linear(self.fc_input_size, 120)
        self.act3 = nn.Sigmoid()

        # Dense2: 120 -> 84
        self.fc2 = nn.Linear(120, 84)
        self.act4 = nn.Sigmoid()

        # Dense3: 84 -> 10
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))

        x = x.view(-1, self.fc_input_size)

        x = self.act3(self.fc1(x))
        x = self.act4(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
def train_model(epochs=10, lr=0.001):
    train_loader, _ = load_mnist(batch_size=64)

    model = LeNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

    return model

In [4]:
def evaluate_model(model):
    _, test_loader = load_mnist(batch_size=1000)

    model.eval()

    all_preds = []
    all_labels = []
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    return accuracy, precision, recall, f1

trained_model = train_model(epochs=10, lr=0.001)

accuracy, precision, recall, f1 = evaluate_model(trained_model)

print(f"Độ chính xác (Accuracy): {accuracy:.2f}%")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")
print(f"F1-score (Macro): {f1:.4f}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 523kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.81MB/s]


Epoch [1/10], Loss: 0.9342
Epoch [2/10], Loss: 0.2417
Epoch [3/10], Loss: 0.1550
Epoch [4/10], Loss: 0.1158
Epoch [5/10], Loss: 0.0909
Epoch [6/10], Loss: 0.0780
Epoch [7/10], Loss: 0.0657
Epoch [8/10], Loss: 0.0594
Epoch [9/10], Loss: 0.0535
Epoch [10/10], Loss: 0.0487
Độ chính xác (Accuracy): 98.48%
Precision (Macro): 0.9846
Recall (Macro): 0.9848
F1-score (Macro): 0.9847
